In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
"""
Copyright (c) Meta Platforms, Inc. and affiliates.

This source code is licensed under the CC BY-NC license found in the
LICENSE.md file in the root directory of this source tree.
"""
import pickle
import random
import time
import gym
import numpy as np

import sys
import os
from munch import Munch
import yaml

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from PIL import Image
import os
import pandas as pd


from IPython import display
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from PIL import Image


device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)


cuda:1


In [3]:
df = pd.read_csv('ema.csv', sep=',')
df = df.sort_values(['subid', 'dttm_obs'])[['dttm_obs', 'subid', 'ema_1', 'ema_2', 'ema_3', 'ema_4', \
                                            'ema_5', 'ema_6', 'ema_7', 'ema_8', 'ema_9', 'ema_10']]

# Handling NA: drop the entire row if 4th entry is NA 
# For morning-only survey, set dummy=1 if survey reported, otherwise dummy=0
df = df.fillna(-6.0)
df = df[df['ema_4'] > -1]
df['ema_dummy'] = (df['ema_8'] > -1)
df.loc[df['ema_8'] < 0, ['ema_8', 'ema_9', 'ema_10']] = 0

# Encoding: 'Yes/No -> 0/1'
df = df.replace('No',0)
df = df.replace('Yes',1)

# Date-time encoding -- in hour unit 
df['date'] = (pd.to_datetime(df['dttm_obs']).astype('int64') // (10**9)) / 3600
df2 = df[['subid', 'ema_1', 'ema_2', 'ema_3', 'ema_4', \
            'ema_5', 'ema_6', 'ema_7', 'ema_8', 'ema_9', 'ema_10', 'ema_dummy', 'date']]


time_df = pd.read_csv('labels_1day.csv', sep=',')
time_df = time_df.sort_values(['subid', 'dttm_label']) 
time_df['date'] = (pd.to_datetime(time_df['dttm_label']).astype('int64') // (10**9)) / 3600
time_df = time_df.replace('no',0)
time_df = time_df.replace('yes',1)

time_df = time_df[['subid', 'lapse', 'date']]

In [4]:
def input_data(org_data, time_label):
    # 24hr window right-index
    fj = 0    
    
    # offset starting date-time to 0 
    offset_time = org_data[0,-1] + 0.0
    org_data[:,-1] -= offset_time
    time_label[:,-1] -= offset_time
    feature = np.median(org_data[:, 1:-5], axis=0)
    
    # print(feature.shape)
    dataset = []
    tq = ts = 0
    while(tq < time_label.shape[0] and ts < org_data.shape[0] ):
        t_feature = np.concatenate([feature, np.median(org_data[ts:, 1:-5], axis=0)])
        
        if(ts == org_data.shape[0] or (tq < time_label.shape[0] and time_label[tq, -1] < org_data[ts, -1])):
            y = 1 if( time_label[tq,1] > 0 ) else 0
            dataset.append({'type':'query', 'out': y, 'time': time_label[tq,-1], 'feature': t_feature})
            while(tq < time_label.shape[0] and time_label[tq,-1] < org_data[ts,-1] ):
                tq += 1
        else:
            y = 1 if( tq < time_label.shape[0] and time_label[tq,1] > 0 ) else 0
            dataset.append({'type':'survey', 'obs': org_data[ts, 1:-1]+0.0, 'time': org_data[ts,-1], 
                            'out':y, 'feature': t_feature})
            ts += 1

    return dataset


In [5]:
raw_data = df2.values.astype(np.float32)
time_raw_data = time_df.values.astype(np.float32)

env_list, subid_list = [], []
print_list = []
for subid in range(270):
    sub_data = raw_data[raw_data[:,0]==(subid+1)]
    time_label = time_raw_data[time_raw_data[:,0]==(subid+1)]
    if( len(sub_data) == 0 or len(time_label) == 0 ):
        continue
          
    subid_list.append(subid)
    print_list.append(subid+1)
    env_list.append(input_data(sub_data, time_label))

print(print_list)

[1, 2, 3, 5, 6, 7, 9, 10, 11, 15, 16, 18, 19, 20, 21, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 37, 38, 39, 40, 42, 43, 44, 47, 48, 51, 52, 53, 54, 56, 58, 59, 63, 64, 65, 66, 74, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 88, 90, 92, 93, 94, 97, 98, 99, 100, 103, 109, 110, 116, 117, 118, 119, 121, 128, 130, 131, 134, 135, 136, 137, 138, 139, 143, 149, 150, 156, 158, 161, 162, 163, 166, 167, 169, 171, 172, 175, 178, 179, 180, 181, 183, 185, 187, 188, 189, 190, 191, 192, 193, 196, 197, 200, 201, 203, 205, 207, 208, 209, 211, 212, 213, 214, 215, 218, 221, 222, 223, 224, 225, 230, 231, 232, 234, 236, 238, 240, 241, 242, 243, 245, 248, 252, 255, 259, 262, 263, 264, 265, 268, 270]


In [ ]:
import xgboost as xgb
from sklearn import metrics
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def input_xgboost_data(org_data, time_label):
    j = [0,0,0,0,0]
    fj = 0
    window = [12, 24, 48, 72, 168]
    
    dataset = []
    tq = ts = 0

    while( ts < org_data.shape[0] ):
        while(tq < time_label.shape[0] and time_label[tq,-1] < org_data[ts,-1] ):
            tq += 1
            
        if( tq >= time_label.shape[0] ):
            break
            
        time = time_label[tq,-1]
        while( ts < org_data.shape[0] and org_data[ts,-1] <= time ):
            ts += 1
        
        for k in range(5):
            while( j[k] < ts-1 and org_data[j[k], -1] < time - window[k] ):
                j[k] = j[k] + 1
                               
        X1 = np.concatenate(([np.mean(org_data[j[k]:ts, 1:], axis=0) for k in range(5)], \
                            [np.min(org_data[j[k]:ts, 1:], axis=0) for k in range(5)] , \
                            [np.max(org_data[j[k]:ts, 1:], axis=0) for k in range(5)]), axis=1)
        X1 = np.concatenate(X1)

        sub_mean = np.mean(org_data[:ts], axis=0)
        X = np.concatenate((X1, sub_mean[1:]))
        X = np.concatenate((X, org_data[ts-1, 1:]))

        y = 1 if (time_label[tq,1] > 0 ) else 0        
        dataset.append({'in':X, 'out':y})

    return dataset


def xgboost_main(train_index, test_index):
    trainset = []
    for m in train_index:
        subid = subid_list[m] + 1
        sub_data = raw_data[raw_data[:,0] == subid] 
        time_label = time_raw_data[time_raw_data[:,0]==subid]                
        trainset.extend(input_xgboost_data(sub_data, time_label))

    train_loader = torch.utils.data.DataLoader(trainset, batch_size=500000, shuffle=True, drop_last=False)

    testset = []
    for m in test_index:
        subid = subid_list[m] + 1
        sub_data = raw_data[raw_data[:,0] == subid]     
        time_label = time_raw_data[time_raw_data[:,0]==subid]                
        testset.extend(input_xgboost_data(sub_data, time_label))

    test_loader = torch.utils.data.DataLoader(testset, batch_size=60000, shuffle=False, drop_last=False)

    data_iter = iter(train_loader)
    train_data = next(data_iter)
    X = train_data['in']
    y = train_data['out']

    test_iter = iter(test_loader)
    test_data = next(test_iter)

    # Create regression matrices
    dtrain_reg = xgb.DMatrix(X, y)

    X_test = test_data['in']
    y_test = test_data['out']
    dtest_reg = xgb.DMatrix(X_test, y_test)

    params = {"objective": "reg:squarederror", "tree_method": "hist", "device": "cuda"}

    print(X.shape, X_test.shape)
    
    n = 10
    xgmodel = xgb.train(
       params=params,
       dtrain=dtrain_reg,
       num_boost_round=n,
    )

    preds = xgmodel.predict(dtest_reg)
    print(len(y_test[y_test>0]), len(preds[preds>0.1]), len(preds))
    print('XgBoost: AUC = %.2f' % roc_auc_score(y_test, preds))  #1


torch.Size([29658, 187]) torch.Size([7558, 187])
420 1619 7558
XgBoost: AUC = 0.90
tensor(0.2216, grad_fn=<MeanBackward0>)
myFCN: AUC = 0.85
---------------------------------------------------
torch.Size([29936, 187]) torch.Size([7280, 187])
643 1589 7280
XgBoost: AUC = 0.86
tensor(0.2712, grad_fn=<MeanBackward0>)
myFCN: AUC = 0.81
---------------------------------------------------
torch.Size([30143, 187]) torch.Size([7073, 187])
699 2045 7073
XgBoost: AUC = 0.86
tensor(0.2087, grad_fn=<MeanBackward0>)
myFCN: AUC = 0.83
---------------------------------------------------
torch.Size([30012, 187]) torch.Size([7204, 187])
693 1777 7204
XgBoost: AUC = 0.89
tensor(0.2027, grad_fn=<MeanBackward0>)
myFCN: AUC = 0.85
---------------------------------------------------
torch.Size([29891, 187]) torch.Size([7325, 187])
685 1904 7325
XgBoost: AUC = 0.87
tensor(0.2349, grad_fn=<MeanBackward0>)
myFCN: AUC = 0.78
---------------------------------------------------
torch.Size([29962, 187]) torch.Size

# Environment

In [ ]:
from env import Env, VecEnv

# parallel env runs
num_process = 50
# survey window size (last "window_size" survey received points)
window_size = 30


# action encoding: 
# 0 - no survey or predict
# 1 - yes survey or predict
action_dim = 1
action_max = 2

# feature encoding: mean of survey items
feature_dim = 14

# y encoding
# 0 - no lapse within 24hr
# 1 - yes lapse within 24hr
state_dim, embed_dim, y_max = 11, 128, 2

# Reinforcement Learning

In [ ]:
from env import Env, VecEnv

# parallel env runs
num_process = 50
# survey window size (last "window_size" survey received points)
window_size = 30


# action encoding: 
# 0 - no survey or predict
# 1 - yes survey or predict
action_dim = 1
action_max = 2

# feature encoding: mean of survey items
feature_dim = 14

# y encoding
# 0 - no lapse within 24hr
# 1 - yes lapse within 24hr
state_dim, embed_dim, y_max = 11, 128, 2

In [ ]:

# get trajectory sample batch
def get_sample(traj, model, device):
    b1 = model(traj['prev_state'], traj['prev_action'], output_embedding=True).detach()
    action = traj['action'].reshape(-1, 1).to(device)  # [B, n]
    reward = traj['reward'].reshape(-1, 1).to(device)  # [B, n]
    b2 = model(traj['next_state'], traj['next_action'], output_embedding=True).detach()
    t1 = traj['ptype']['type']
    t2 = traj['ntype']['type']
    done = traj['done'].reshape(-1, 1).to(device)

    return t1, b1, action, reward, t2, b2, done

# train function (without extra features)
def train(venv, model, agent, feature_optimizer, predict_buffer, max_iter = 50, freq=1, burn_in = 0, feature_weight = 0.1):
    buffer = ReplayBuffer()
    ce_criterion = torch.nn.CrossEntropyLoss(reduction='none')
    mse_criterion = torch.nn.MSELoss(reduction='none')
    
    for episode in range(max_iter):
        obs, info = venv.reset(mode='train')

        # initialize slideing window
        ts = torch.zeros(num_process, 500, state_dim).to(device) 
        ta = torch.zeros(num_process, 500, action_dim).to(int).to(device) 
        weights = torch.zeros(num_process, 500, 1).to(device)
        steps = torch.zeros(num_process, 1).to(int).to(device)
        # initialize belief states
        bf = model(ts, ta, output_embedding=True, steps=steps).detach()

        # initialize buffers / debug variables
        sample_traj = []
        predict_samples = []
        
        measured_ta = []
        measured_tr = []
        cumr = []
        
        # generate trajectories
        for t in range(1000):
            p = random.random()
            
            # epsilon greedy
            if( t < 5 or episode < burn_in ):
                actions = torch.zeros(num_process,action_dim).to(int) + 1
            elif( 0.45 < p and p < 0.5 ):
                actions = torch.randint(low=0, high=action_max, size=(num_process,action_dim))
            # burn-in data
            else:
                actions, _, _ = agent.select_action(bf, types)
                actions = actions.detach().cpu()

            obs, rewards, dones, infos = venv.step(actions)        
            all_done = True
            #predict_ys, _ = model.predict_forward(bf)            
            #check_y = nn.Softmax(dim=-1)(predict_ys[11]).reshape(y_max).detach().cpu()                
            
            for i in range(num_process):
                if( infos[i] == -1 ):
                    continue
                
                all_done = False
                # ptype = {'type': types[i], 'total_collect': old_cnt[i], 'recent_collect': rec_cnt[i]}
                
                if (infos[i]['type'] == 'survey'):
                    ts[steps[i]] = obs[i].reshape(1,-1).to(device)
                    ta[steps[i]] = (actions[i]%2).reshape(1,-1).to(device)
                    steps[i] += 1

                else:
#                     if( i == 11 and (t+1) % 50 == 0 ):
#                         print(pts[0], actions[i], check_y[1], infos[i]['out'], rewards[i])
                    
                    predict_samples.append({'action': pta, 'state': pts, 'out': infos[i]['out'], \
                                            'feature': infos[i]['feature']})

#                 types[i] = 1 if( infos[i]['next_type'] == 'query' ) else 0
#                 ntype = {'type': types[i], 'total_collect': old_cnt[i], 'recent_collect': rec_cnt[i]}
#                 nts, nta = ts[i] + 0.0, ta[i] + 0

                sample_traj.append({'ptype': ptype, 'prev_state': pts, 'prev_action': pta, \
                        'action': actions[i], 'reward': rewards[i], \
                        'ntype': ntype, 'next_action': nta, 'next_state': nts, 'done': dones[i]})

                if(i==11):
                    measured_tr.append(rewards[i].item())
                    measured_ta.append(actions[i].item()%2)
                    cumr.append(rewards[i])
                    
            if( all_done ): 
                break
                
            bf = model(ts, ta, output_embedding=True, steps=steps).detach()

        # print('episode ', episode, '- reward average:', ta[11].transpose(0,1), sum(cumr), len(cumr) )#, measured_ta, measured_tr)
        buffer.add_episodes(sample_traj)
        predict_buffer.add_episodes(predict_samples)
        
        #######################################################
        # Training Phase        
        #######################################################
        # 1. train prediction first
        feat_train_loaders = predict_buffer.get_loader()
        feat_data_iter = iter(feat_train_loaders)

        for j in range(0, 200):
            try:
                traj = next(feat_data_iter)

                refs = model(traj['state'], traj['action'])
                out_ys, out_features = model.predict_forward(refs)
                ys = traj['out'].to(device).reshape(-1)
                features = traj['feature'].to(device).reshape(-1, feature_dim)

                predict_loss = ce_criterion(out_ys, ys).mean()
                # predict_loss += feature_weight * mse_criterion(out_features, features).mean()
                predict_loss.backward()
                
                feature_optimizer.step()
                feature_optimizer.zero_grad()

            except StopIteration:
                feat_train_loaders = predict_buffer.get_loader()
                feat_data_iter = iter(feat_train_loaders)
                continue
        
        print(predict_loss)
        continue
                        
        # 2. then train q
        train_loaders = buffer.get_loader()
        data_iter = iter(train_loaders)

        for j in range(0, 500):
            try:
                traj = next(data_iter)
                batch = get_sample(traj, model, device)

            except StopIteration:
                train_loaders = buffer.get_loader()
                data_iter = iter(train_loaders)
                continue

            agent.update(batch)

        agent.update_target()



In [ ]:

def test(venv, model, agent, mode='policy', run_id=0, kfold_id=0, penalty=-1., freq=1):
    scatter = []
    num_process = len(venv.test_ids)
    
    cnt, n_measure, n_hit = [0 for i in range(num_process)], [0 for i in range(num_process)], [0 for i in range(num_process)]
    y, outy = [], []

    obs, info = venv.reset(mode='test')
    
    ts = torch.zeros(num_process, window_size, state_dim).to(device)
    types = torch.zeros(num_process).to(device)
    ta = torch.zeros(num_process, window_size, action_dim).to(int).to(device)
    old_cnt = [0 for t in range(num_process)]
    rec_cnt = [0 for t in range(num_process)]

    bf = model(ts, ta, output_embedding=True).detach()    
    for t in range(1000):
#         if( mode == 'policy' ):
#             actions, _, _ = agent.select_action(bf, types)
#             actions = actions.detach().cpu()
#         else:

        p = random.random()
        actions = torch.zeros(num_process, action_dim).to(int)
        if( p < 1. / freq ):
            actions = actions + 1
            
        obs, rewards, dones, infos = venv.step(actions)
        predict_ys, _ = model.predict_forward(bf)
        predict_ys = predict_ys.detach()
   
        # add transition data to dataset
        all_done = True
        for i in range(num_process):
            if( infos[i] == -1 ):
                continue

            all_done = False
            if (infos[i]['type'] == 'query'):
                check_y = nn.Softmax(dim=-1)(predict_ys[i]).reshape(y_max).detach().cpu()                
                y.append(infos[i]['out'])
                outy.append(check_y[1])
                
            else:
                ts[i] = torch.cat([ts[i][1:], obs[i].reshape(1,-1).to(device)], dim=0)
                ta[i] = torch.cat([ta[i][1:], (actions[i]%2).reshape(1,-1).to(device)], dim=0)
                
                cnt[i] += 1
                n_measure[i] += 1 if( actions[i] % 2 == 1 ) else 0
                n_hit[i] += 1 if( obs[i,0] > 0 ) else 0

            types[i] = 1 if( infos[i]['next_type'] == 'query' ) else 0

        if( all_done ): 
            break
                
        bf = model(ts, ta, output_embedding=True).detach()

    for i in range(num_process):
        scatter.append([n_measure[i]/cnt[i], n_hit[i]/cnt[i]])
        print(venv.test_ids[i], ':', n_measure[i], n_hit[i], cnt[i])
    
    path = os.path.join("./results", f"run_{run_id}_kfold_{kfold_id}_penalty_{penalty}.pt")
    things_to_save = {
        'test_ids': venv.test_ids,
        'model_state_dict': model.state_dict(),
        'agent_state_dict': agent.save_dict()
    }
    torch.save(things_to_save, path)
    
    scatter = np.array(scatter)
    import matplotlib.pyplot as plt
    plt.figure(figsize=(4, 3))  

    plt.scatter(scatter[:,1], scatter[:,0])
    plt.xlabel("Average Lapse Count")
    plt.ylabel("Survey Ratio")
    plt.show()

    print('Average Survey Ratio:', np.mean(scatter[:,0]))

    import xgboost as xgb
    from sklearn import metrics
    from sklearn.metrics import roc_auc_score
    import matplotlib.pyplot as plt
    from sklearn.model_selection import train_test_split

    print('AUC = %.2f' % roc_auc_score(y, outy))  #1

    
    

# Test

In [ ]:
import random
from SACD import init_SACD_agent
from model import build_model
from buffer import ReplayBuffer
  
import torch.nn.functional as F
import time
import copy

# weight-decay params: {1e-3, 5e-4, 1e-4, 5e-5, 1e-5}

def k_fold_test(k=5, run_id=0):
    shuffled_index = list(range(151))
    random.shuffle(shuffled_index)

    for i in range(k):
        st,et = i*151//k, (i+1)*151//k

        test_index = shuffled_index[st:et]
        train_index = list(set(range(151)) - set(test_index))
        xgboost_main(train_index, test_index)
        venv = VecEnv(env_list, train_index, test_index)
        
        model = build_model(embed_dim, state_dim, action_dim, feature_dim, action_max, \
                            y_max, model_family='gpt2').to(device)
        agent = init_SACD_agent(embed_dim, action_dim, action_max, device, weight_decay=1e-4, \
                                lr_actor=1e-4, lr_critic=2e-4, entropy_regularizer=0.03)
        feature_optimizer = torch.optim.Adam([{'params': model.parameters(), 'lr': 5e-5, 'weight_decay': 3e-4}])
        predict_buffer = ReplayBuffer()

        # init predictor part
        venv.set_probe_penalty(0.01)
        train(venv, model, agent, feature_optimizer, predict_buffer, max_iter=30)
        # Test full observation-prediction
        test(venv, model, agent, 'full', run_id=run_id, kfold_id=i, penalty=-1)

        penalty=0.05
        for freq in [2,4,10,20]:
            venv.set_probe_penalty(penalty)
            train(venv, model, agent, feature_optimizer, predict_buffer, max_iter=20, freq=freq)
            test(venv, model, agent, run_id=run_id, kfold_id=i, penalty=penalty, freq=freq)

        print('-----------------------------------------------')

In [ ]:
k_fold_test(5, 1)